# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# for creating loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

#print(full_data_rows_list)


csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
#number of rows
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821



# Part II. The Apache Cassandra coding portion of project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [5]:
# create cluster
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) 
    session = cluster.connect()
except Exception as e:
    print(e)

In [6]:
# keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS events
    with replication = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
)
except Exception as e:
    print(e)

In [7]:
# set keyspace
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [11]:
#For table artist_history I used item in session and session id columns as a composite key, to uniquely identify the rows in the table.
query= "CREATE TABLE IF NOT EXISTS artist_history"
query = query + "(item_number_in_session int,session_id int, \
artist text, song_title text,length_of_the_song float, Primary Key(item_number_in_session,session_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#insert data into table
        query = "INSERT INTO artist_history (item_number_in_session,session_id,artist,song_title,length_of_the_song )"
        query = query + "Values (%s, %s,%s, %s, %s)"
        session.execute(query, (int(line[3]),int(line[8]),line[0], line[9], float(line[5]) ))

In [14]:
query = "select artist,song_title, length_of_the_song from artist_history WHERE session_id = 338 and item_number_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, ',', row.song_title, '/' ,row.length_of_the_song)


Faithless , Music Matters (Mark Knight Dub) / 495.30731201171875


In [15]:
#I used the https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.values.html to get information on making a data frame
result=session.execute(query)
query1= pd.DataFrame(list(result))
print(query1.values)

[['Faithless' 'Music Matters (Mark Knight Dub)' 495.30731201171875]]


In [16]:
#For table artist_song I used the item_number_in_session as the clustering key to sort out the partition key.
#for the partition key i used the user_id, session_id . 

query= "CREATE TABLE IF NOT EXISTS artist_song"
query= query+"(user_id int,session_id int,item_number_in_session int,artist text, song_title text,\
first_Name_of_user text, last_name_of_user text, Primary Key((user_id, session_id),item_number_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
#Insert data into table
        query = "INSERT INTO artist_song ( user_id,session_id,item_number_in_session,artist,\
        song_title,first_Name_of_user, last_name_of_user)"
        query = query + "Values (%s, %s, %s,%s, %s, %s, %s)"
        session.execute(query,(int(line[10]),int(line[8]),int(line[3]),line[0], line[9],line[1], line[4]))

In [20]:
query = "select artist, song_title,first_name_of_user, last_name_of_user  from artist_song WHERE user_id = 10 and session_id = 182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist,',', row.song_title,'/', row.first_name_of_user, row.last_name_of_user )

Down To The Bone , Keep On Keepin' On / Sylvie Cruz
Three Drives , Greece 2000 / Sylvie Cruz
Sebastien Tellier , Kilometer / Sylvie Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) / Sylvie Cruz


In [21]:
result=session.execute(query)
query2= pd.DataFrame(list(result))
print(query2)

              artist                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name_of_user last_name_of_user  
0             Sylvie              Cruz  
1             Sylvie              Cruz  
2             Sylvie              Cruz  
3             Sylvie              Cruz  


In [22]:
# For table music_app_history I selected the composite partition key(userid & song_title) because the.....
#..song title will filter the column and the user_id  will return all the users who have listened to the specific song.
query= "CREATE TABLE IF NOT EXISTS music_app_history"
query= query+ "(song_title text,user_id int,artist text, first_Name_of_user text, \
last_name_of_user text,  primary key(song_title,user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                     

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#Insert data into table
        query = "INSERT INTO music_app_history (song_title, user_id,artist,first_Name_of_user, last_name_of_user)"
        query = query + "Values (%s, %s,%s, %s, %s)"
        session.execute(query,(line[9],int(line[10]),line[0],line[1], line[4] ))

In [25]:
query = "select first_name_of_user, last_name_of_user from music_app_history WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name_of_user, row.last_name_of_user )

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [26]:
result=session.execute(query)
query3= pd.DataFrame(list(result))
print(query3)

  first_name_of_user last_name_of_user
0         Jacqueline             Lynch
1              Tegan            Levine
2               Sara           Johnson


In [27]:
#drop tables
query = "drop table artist_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [28]:
query = "drop table artist_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [29]:
query = "drop table music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [30]:
#end session
session.shutdown()
cluster.shutdown()